In [1]:
%%writefile going_modular/data_setup.py

import os

import torch
import torch.utils.data.DataLoader as DataLoader
import torch.nn as nn
import torchvision.transform as transform
import torchvision.dataset.ImageFolder as ImageFolder
from typing import Tuple, List

NUM_WORKERS = os.cpu_count()

def create_dataloader(train_dir: str,
                      test_dir: str,
                      train_transform: transform.Compose | nn.Module = None,
                      test_transform: transform.Compose | nn.Module = None,
                      batch_size: int, 
                      num_workers: int = NUM_WORKERS) -> Tuple[DataLoader, DataLoader, List[str]]:
    """ Create train and test DataLoader
    Pass train_dir and test_dir directories to get train and validation DataLoader 
    Args:
        train_dir: Path to training directory.
        test_dir: Path to testing directory.
        transform: torchvision transforms to perform on training and testing data.
        batch_size: Number of samples per batch in each of the DataLoaders.
        num_workers: An integer for number of workers per DataLoader.

    Returns:
        A tuple of (train_dataloader, test_dataloader, class_names).
        Where class_names is a list of the target classes.
    Example usage:
          train_dataloader, test_dataloader, class_names = \
            = create_dataloaders(train_dir=path/to/train_dir,
                                 test_dir=path/to/test_dir,
                                 transform=some_transform,
                                 batch_size=32,
                                 num_workers=4)
    """
    
    
    # Create dataset using ImageFolder
    
    # Train Dataset from train_dir directory
    train_dataset = ImageFolder(root=train_dir,
                                transform=train_transform,
                                target_transform=None)

    # Test Dataset from test_dir directory
    test_dataset = ImageFolder(root=test_dir,
                               transform=test_transform,
                               target_transform=None)
    
    # Get class names of target
    class_names = train_dataset.classes
    
    # Create train, test DataLoader
    train_dataloader = DataLoader(dataset=train_dataset, 
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=num_workers,
                                  pin_memory=True)
    
    test_dataloader = DataLoader(dataset=test_dataset, 
                                  batch_size=batch_size,
                                  shuffle=False,
                                  num_workers=num_workers,
                                  pin_memory=True)
    
    return train_dataloader, test_dataloader, class_names


Writing going_modular/data_setup.py


In [2]:
%%writefile going_modular/model.py

import torch
import torch.nn as nn

class TinyVGG(nn.Module):
    """
    Model architecture copying TinyVGG from: 
    https://poloclub.github.io/cnn-explainer/
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                      out_channels=hidden_units, 
                      kernel_size=3, # how big is the square that's going over the image?
                      stride=1, # default
                      padding='same'), # options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number 
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) # default stride value is same as kernel_size
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) # default stride value is same as kernel_size
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            # Where did this in_features shape come from? 
            # It's because each layer of our network compresses and changes the shape of our inputs data.
            nn.Linear(in_features=hidden_units*16*16,
                      out_features=output_shape)
        )
    
    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x) 
        # print(x.shape)
        x = self.conv_block_2(x) 
        # print(x.shape)
        x = self.classifier(x)
        # print(x.shape)
        return x
        # return self.classifier(self.conv_block_2(self.conv_block_1(x))) # <- leverage the benefits of operator fusion



Writing going_modular/model.py


In [3]:
%%writefile going_modular/engine.py

import torch
import torch.nn as nn
from typing import Dict


def train_step(model: nn.Module,
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = args.device) -> Dict[str, float]:
    
    model.train()
    total_loss, total_acc = 0, 0
    for idx, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        y_pred = model(X)
        
        loss = loss_fn(y_pred, y)
        acc = accuracy_fn(y_pred.argmax(dim = 1), y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss
        total_acc += acc
        
        if idx % 4 == 0:
            print(f"Trained on {idx * len(X)}/{len(dataloader.dataset)}")
            
    total_loss /= len(dataloader)
    total_acc /= len(dataloader)
    
    print(f"Train loss {total_loss:.4f} | Train accuracy {total_acc:.4f}")
    
    return {"loss_score": total_loss, "acc_score" :total_acc}
    
    
    
def test_step(model: nn.Module, 
              dataloader: torch.utils.data.DataLoader,
              loss_fn: nn.Module,
              accuracy_fn,
              device: torch.device = args.device) -> Dict[str, float]:
    model.eval()
    total_loss, total_acc = 0, 0 
    with torch.inference_mode():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            
            y_pred = model(X)
            loss = loss_fn(y_pred, y)
            acc = accuracy_fn(y_pred.argmax(dim = 1), y)
            
            total_loss += loss
            total_acc += acc
            
        total_loss /= len(dataloader)
        total_acc /= len(dataloader)
    
    print(f"Test loss {total_loss:.4f} | Test accuracy {total_acc:.4f}")
    
    return {"loss_score": total_loss, "acc_score": total_acc}



def train_loop(model: nn.Module, 
               train_loader: torch.utils.data.DataLoader,
               test_loader: torch.utils.data.DataLoader,
               loss_fn: nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               epochs: int = args.epochs) -> Dict[str, float]:

    result = {
        "loss_train": [],
        "acc_train": [],
        "loss_test": [],
        "acc_test": []
    }
    start_train = timer()

    for epoch in tqdm(range(epochs)):
        print(f"Epoch {epoch}:\n-----------\n")

        train_score = train_step(model=model, 
                   dataloader=train_loader,
                   loss_fn=loss_fn,
                   accuracy_fn=accuracy_fn,
                   optimizer=optimizer)

        test_score = test_step(model=model,
                  dataloader=test_loader,
                  loss_fn=loss_fn,
                  accuracy_fn=accuracy_fn)
        result["loss_train"].append(train_score["loss_score"].to("cpu").detach().numpy())
        result["acc_train"].append(train_score["acc_score"].to("cpu").detach().numpy())
        result["loss_test"].append(test_score["loss_score"].to("cpu").detach().numpy())
        result["acc_test"].append(test_score["acc_score"].to("cpu").detach().numpy())
        
    end_train = timer()

    time_train = print_train_time(start=start_train,
                                     end=end_train)    
    result["total_time"] = time_train
    return result
    

Writing going_modular/engine.py
